**Checkpoint II**

In [68]:
import pandas as pd
from datetime import date, timedelta
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

In [69]:
training_data_df = pd.read_csv("train.csv")
training_data_df['date'] = pd.to_datetime(training_data_df['date'], format='%Y-%m-%d')

#remove entries bewtween the day before the earthquake struck and 3 months after.
training_data_df = training_data_df.loc[(training_data_df['date'] < '2016-04-15')
                     | (training_data_df['date'] > '2016-05-15')]
                     
id_col = training_data_df[["id"]]
target = "sales"
y_data = training_data_df[[target]]
X_data = training_data_df.drop(target, axis=1)
X_data

,id,date,store_nbr,family,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0
1,1,2013-01-01,1,BABY CARE,0
2,2,2013-01-01,1,BEAUTY,0
3,3,2013-01-01,1,BEVERAGES,0
4,4,2013-01-01,1,BOOKS,0
...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,1
3000885,3000885,2017-08-15,9,PRODUCE,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,8


In [70]:
x_test = pd.read_csv("test.csv")
id_col = x_test[["id"]]
x_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [71]:
numeric_features = ["store_nbr", "onpromotion"]
categorical_features = ["family"]
drop_features = ["id", "date"]
transformer = make_column_transformer((StandardScaler(), numeric_features), (OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_features), ("drop", drop_features), n_jobs=-1)

In [72]:
pipeline = make_pipeline(transformer)
pipeline.fit_transform(X_data, y_data)

array([[-1.70026736, -0.21342588,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.70026736, -0.21342588,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.70026736, -0.21342588,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.1869791 , 12.18646359,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [-1.1869791 ,  0.45683842,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-1.1869791 , -0.21342588,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

## Linear Regressor Method

In [73]:
regressor = LinearRegression()
regressor.fit(pipeline.transform(X_data), y_data)

LinearRegression()

In [74]:
results = regressor.predict(pipeline.transform(x_test))
id_column = []
for i in range(len(results)):
    id_column.append(3000888 + i)

results = pd.DataFrame(results, columns=['sales'])
results.insert(0, 'id', id_column, True)

# print(results.reset_index().to_string(index=False))
# results.index.name = None

# results.columns.name = {'id' : 'sales'}

# df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
# results.columns.names = {'id' : 'sales'}
# results.index.names = ['id']
#results.drop('')
#results.rename(columns={'':'id'}, inplace=True)#.columns = ["id", "sales"]#.rename(columns={0:'id', 1:'sales'}) # .replace(0, id_col)
results.to_csv('predictions2.csv', index=False)
results

,id,sales
0,3000888,-70.453857
1,3000889,-76.009644
2,3000890,-34.333374
3,3000891,2490.097168
4,3000892,-75.137451
...,...,...
28507,3029395,268.957275
28508,3029396,36.388306
28509,3029397,1062.317139
28510,3029398,123.655518
